In [3]:
f = open("weather.txt")
out = open("weather.csv", "w")
headers = [
    "YR", "MT", "DY", "TEMP MAX", "TEMP MIN", 
    "TEMP AVG", "TEMP DEP", "TEMP HDD", 
    "TEMP CDD", "PCPN WTR", "SNOW SNW", 
    "SNOW DPTH", "WIND AVG SPD", "WIND MX SPD", 
    "WIND 2MIN DIR", "SUNSHINE MIN", "SUNSHINE PSBL", 
    "SKY S-S", "SKY WX", "PK SPD", 
    "PK DR"
]
out.write(",".join(headers) + "\n")
content = f.readlines()
f.close()
date = ["", "", ""]
month_d = {"january": "01", 
           "february": "02", 
           "march": "03", 
           "april": "04", 
           "may": "05", 
           "june": "06", 
           "july": "07", 
           "august": "08", 
           "september": "09", 
           "october": "10", 
           "november": "11", 
           "december": "12"}
for line in content:
    line = line.replace("\n", "").strip()
    while "  " in line:
        line = line.replace("  ", " ")
    if len(line) == 0 or line.startswith("PRELIMINARY"):
        continue
    if line.startswith("LATITUDE") or line.startswith("LONGITUDE"):
        continue
    if line.startswith("TEMPERATURE") or line.startswith("======"):
        continue
    if line.startswith("1 2 3 4 5 6A") or line.startswith("12Z AVG MX 2MIN"):
        continue
    if line.startswith("DY MAX MIN AVG"):
        continue
    if line.startswith("STATION"):
        if line.split(" ")[1] != "MADISON":
            print("ERROR")
            break
        continue
    if line.startswith("MONTH"):
        date[1] = month_d[line.split(" ")[1].lower()]
        continue
    if line.startswith("YEAR"):
        date[0] = line.split(" ")[1]
        continue
        
    temp = line.split(" ")
    if len(temp) == 18:
        temp = temp[:16] + [""] + temp[16:]
    if len(temp[0]) == 1:
        date[2] = "0" + temp[0]
    else:
        date[2] = temp[0]
    temp[0] = "/".join(date)
    temp = [date[0], date[0] + "/" + date[1]] + temp
    for idx in [9,10,11, 12, 13]:
        if temp[idx] == "T":
            temp[idx] = "0"
        if temp[idx] == "M":
            temp[idx] = "0"
    out.write(",".join(temp) + "\n")
out.close()

In [143]:
files = ["2015-Table 1.csv", "2016-Table 1.csv", "2017-Table 1.csv", 
        "2018-Table 1.csv", "2019-Jan-March5-Table 1.csv"]
content_cost = []
for file in files:
    f = open(file)
    data = list(csv.reader(f))
    csv_header_cost = data[0]
    csv_lines = data[1:]
    for line in csv_lines:
        line[7] = line[7].replace(",", "")
        line[15] = line[15].replace(",", "")
        temp = line[8].split("/")
        try:
            line[8] = "/".join([temp[2], temp[0], temp[1]])
        except:
            print(line)
    content_cost += csv_lines
    f.close()
    

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [144]:
content_cost[0][8]

'2015/12/30'

In [145]:
import csv
f = open("weather.csv")
data = list(csv.reader(f))
csv_header_weather = data[0]
csv_lines = data[1:]
f.close()
weather_d = {}

for row in csv_lines:
    weather_d[row[2]] = row


In [146]:
f = open("combined.csv", "w")
headers = csv_header_cost + csv_header_weather
csv_writer = csv.writer(f)
csv_writer.writerow(headers)
for row in content_cost:
    try:
        csv_writer.writerow(row + weather_d[row[8]])
    except:
        pass
    
f.close()

In [125]:
a = "a  s     s"
while "  " in a:
    a = a.replace("  ", " ")
a.split(" ")

['a', 's', 's']

In [127]:

import pandas as pd
import sqlite3

conn = sqlite3.connect('weather_cost.db')
df = pd.read_csv('combined-8.csv')
df.to_sql('data', conn, if_exists = 'replace', index = False)
conn.close()

c = sqlite3.connect('weather_cost.db')
sql = '''
select "YR", "MT", "DY", "TEMP MAX", "TEMP MIN",
       "TEMP AVG", "TEMP DEP", "TEMP HDD", "TEMP CDD", "PCPN WTR", "SNOW SNW",
       "SNOW DPTH", "WIND AVG SPD", "WIND MX SPD", "WIND 2MIN DIR",
       "SUNSHINE MIN", "SUNSHINE PSBL", "SKY S-S", "SKY WX", "PK SPD",
       "PK DR" from data 
group by DY
order by DY
'''
df = pd.read_sql(sql, c)
c.close()
conn = sqlite3.connect('weather.db')
df.to_sql('weather', conn, if_exists = 'replace', index = False)
conn.close()

,YR,MT,DY,TEMP MAX,TEMP MIN,TEMP AVG,TEMP DEP,TEMP HDD,TEMP CDD,PCPN WTR,...,SNOW DPTH,WIND AVG SPD,WIND MX SPD,WIND 2MIN DIR,SUNSHINE MIN,SUNSHINE PSBL,SKY S-S,SKY WX,PK SPD,PK DR
0,2015,01/2015,01/01/2015,31,14,23,4,42,0,0.00,...,0,7.4,15,240,M,M,5,NaN,24,240
1,2016,01/2016,01/01/2016,28,17,23,4,42,0,0.00,...,2,7.9,15,230,M,M,8,NaN,22,220
2,2017,01/2017,01/01/2017,36,17,27,8,38,0,0.00,...,3,5.3,14,190,M,M,4,NaN,17,210
3,2018,01/2018,01/01/2018,2,-13,-5,-24,70,0,0.00,...,2,6.1,13,310,M,M,0,NaN,19,320
4,2019,01/2019,01/01/2019,30,19,25,6,40,0,0.00,...,4,6.8,17,10,M,M,9,1.0,26,360
5,2016,01/2016,01/02/2016,32,17,25,6,40,0,0.00,...,2,5.7,16,230,M,M,0,NaN,26,250
6,2018,01/2018,01/05/2018,8,-9,0,-19,65,0,0.00,...,2,5.3,13,350,M,M,1,NaN,20,310
7,2016,01/2016,01/08/2016,38,34,36,17,29,0,0.37,...,2,3.2,9,150,M,M,10,1.0,11,130
8,2019,01/2019,01/08/2019,41,19,30,11,35,0,0.00,...,0,11.3,23,310,M,M,9,1.0,41,290
9,2015,01/2015,01/09/2015,9,-1,4,-15,61,0,0.01,...,3,8.9,20,300,M,M,4,8.0,30,310


In [129]:
c = sqlite3.connect('weather_cost.db')
sql = '''
select "YR", "MT", "DY", "ORG", "OBJECT", "PROJECT", "ACCOUNT", "DESCRIPTION", "YEAR", "PER",
       "JOURNAL", "EFF DATE", "SRC", "T", "REF1", "PO/REF2", "REF3",
       "REFERENCE", "AMOUNT" from data 
order by DY
'''
df = pd.read_sql(sql, c)
c.close()
conn = sqlite3.connect('cost.db')
df.to_sql('cost', conn, if_exists = 'replace', index = False)
conn.close()

,YR,MT,DY,ORG,OBJECT,PROJECT,ACCOUNT,DESCRIPTION,YEAR,PER,JOURNAL,EFF DATE,SRC,T,REF1,PO/REF2,REF3,REFERENCE,AMOUNT
0,2015,01/2015,01/01/2015,44301,57140,0,1100-40-44-443-4401-0000-00000-00000-57140-00000,SW ADM ID CHRG EN,2015,1,4423.0,01/01/2015,GEN,1.0,MAINTE,JAN MAINT,None,MMB MAINTE,1059.13
1,2015,01/2015,01/01/2015,44330,53240,0,1100-40-44-443-4430-0000-00000-00000-53240-00000,ST SN PLOW SNOW SUP,2015,1,4416.0,01/01/2015,GNI,1.0,None,REVERSE,2014 ADJ,REV 14 ADJ,-232356.49
2,2015,01/2015,01/01/2015,44331,52410,0,1100-40-44-443-4431-0000-00000-00000-52410-00000,ST SN REMV HLTH INS B,2015,1,1873.0,01/01/2015,GNI,1.0,None,015,None,RECL HLTH,1.45
3,2015,01/2015,01/01/2015,44332,52410,0,1100-40-44-443-4432-0000-00000-00000-52410-00000,ST SN CROS HLTH INS B,2015,1,1873.0,01/01/2015,GNI,1.0,None,015,None,RECL HLTH,1.45
4,2015,01/2015,01/01/2015,44333,52410,0,1100-40-44-443-4433-0000-00000-00000-52410-00000,ST SN BARR HLTH INS B,2015,1,1873.0,01/01/2015,GNI,1.0,None,015,None,RECL HLTH,1.45
5,2015,01/2015,01/01/2015,44301,52410,0,1100-40-44-443-4401-0000-00000-00000-52410-00000,SW ADM HLTH INS B,2015,1,1873.0,01/01/2015,GNI,1.0,None,015,None,RECL HLTH,845.39
6,2015,01/2015,01/01/2015,44334,52410,0,1100-40-44-443-4434-0000-00000-00000-52410-00000,ST SN MISC HLTH INS B,2015,1,1873.0,01/01/2015,GNI,1.0,None,015,None,RECL HLTH,1129.45
7,2015,01/2015,01/01/2015,44330,52410,0,1100-40-44-443-4430-0000-00000-00000-52410-00000,ST SN PLOW HLTH INS B,2015,1,1873.0,01/01/2015,GNI,1.0,None,015,None,RECL HLTH,7204.10
8,2016,01/2016,01/01/2016,44301,52410,0,1100-40-44-443-4401-0000-00000-00000-52410-00000,SW ADM HLTH INS B,2016,1,992.0,01/01/2016,GNI,1.0,None,None,None,RCL PP HTH,4712.91
9,2016,01/2016,01/01/2016,44330,52410,0,1100-40-44-443-4430-0000-00000-00000-52410-00000,ST SN PLOW HLTH INS B,2016,1,992.0,01/01/2016,GNI,1.0,None,None,None,RCL PP HTH,14898.53
